# **Publishing RDF Data to GraphDB Using APIs**

In this section, we'll walk through the steps required to publish RDF data (in Turtle format) from the `rdf_files` directory to a remote GraphDB repository. We'll use the GraphDB API to automate the upload of `.ttl` files. Each file will be uploaded to a specified named graph, where the named graph corresponds to the string `https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/{{file_name_without_extension}}`.

**Step 1: Setup and Requirements**

Before proceeding, ensure that you have the following:
1. **GraphDB repository URL**: The URL of your GraphDB instance.
2. **Repository credentials**: Username and password for accessing the repository (if authentication is required).
3. **GraphDB API Endpoint**: You will need to know the API endpoint for uploading RDF data to the repository (usually `/repositories/{repository}/statements`).
4. **Python Libraries**:
   - Install the `requests` library to handle HTTP requests.
   - Ensure the `os` library is available for directory and file handling.

**Step 2: Writing the Python Script**

This script will iterate over all `.ttl` files in the `rdf_files` directory, and for each file, it will upload the RDF data to a specified named graph in the remote GraphDB repository.

In [1]:
import os
import requests
import urllib.parse

# Configuration
GRAPHDB_BASE_URL = "https://vocabs.ilc4clarin.ilc.cnr.it/graphdb10/repositories"
REPOSITORY_ID = "h2iosc_skosmos"  # Replace with your GraphDB repository ID
HEADERS = {
    "Content-Type": "text/turtle"
}


def read_rdf_file(file_path):
    """
    Function to read the RDF Turtle file and return its content as a string.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

def publish_rdf_file_to_graphdb(file_name, file_content, named_graph_uri):
    """
    Publish an RDF Turtle file to the remote GraphDB repository.
    """
    # Encode the named graph URI to be used in the query parameter
    encoded_graph_uri = urllib.parse.quote(named_graph_uri, safe="")

    # API endpoint for the RDF graph import operation
    url = f"{GRAPHDB_BASE_URL}/{REPOSITORY_ID}/rdf-graphs/service?graph={encoded_graph_uri}"

    # Make the POST request to upload the RDF file with the Turtle content
    response = requests.post(url, headers=HEADERS, data=file_content)

    # Check if the request was successful
    if response.status_code == 204:
        print(f"Successfully submitted {file_name} for import into named graph {named_graph_uri}")
    else:
        print(f"Failed to submit {file_name}. Status code: {response.status_code}, Error: {response.text}")

def publish_all_rdf_files(rdf_directory):
    """
    Iterate over all .ttl files in the rdf_files directory and publish them to GraphDB.
    """
    for filename in os.listdir(rdf_directory):
        if filename.endswith(".ttl"):
            file_path = os.path.join(rdf_directory, filename)
            # Read the file content
            rdf_content = read_rdf_file(file_path)
            
            # Generate the named graph URI based on the file name
            graph_name = filename.replace(".ttl", "")
            named_graph_uri = f"https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/{graph_name}/"

            # Publish the RDF file to the named graph using inline data
            publish_rdf_file_to_graphdb(file_name=filename, file_content=rdf_content, named_graph_uri=named_graph_uri)

# Specify the directory containing .ttl files
rdf_files_directory = "rdf_files"

# Publish all RDF files in the directory to GraphDB
publish_all_rdf_files(rdf_files_directory)

Successfully submitted pl_20_500_11752_OPEN_1018.ttl for import into named graph https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/pl_20_500_11752_OPEN_1018/
Successfully submitted pl_20_500_11752_OPEN_1026.ttl for import into named graph https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/pl_20_500_11752_OPEN_1026/
Successfully submitted pl_20_500_11752_OPEN_1025.ttl for import into named graph https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/pl_20_500_11752_OPEN_1025/
Successfully submitted pl_20_500_11752_OPEN_1016.ttl for import into named graph https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/pl_20_500_11752_OPEN_1016/
Successfully submitted pl_20_500_11752_OPEN_1024.ttl for import into named graph https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/pl_20_500_11752_OPEN_1024/
Successfully submitted pl_20_500_11752_OPEN_1014.ttl for import into named graph https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/pl_20_500_11752_OPEN_1014/
Successfully submitted pl_20_500_11752_OPEN_1019.ttl

### **Publish SKOS Data on SKOSMOS Vocabulary Platform (ILC4CLARIN)**

Publishing SKOS data on a platform like **SKOSMOS** enables easy exploration and access to SKOS-based vocabularies. **SKOSMOS** is an open-source tool designed for publishing SKOS vocabularies and providing a web interface to browse and search vocabularies. For the ILC4CLARIN project, the SKOSMOS platform can be configured to access SKOS vocabularies that have been stored in a triple store, such as **GraphDB**, as described earlier.

In this section, we will go through the necessary steps to publish SKOS data on the **SKOSMOS** platform.

##### **Step 1: Install SKOSMOS (2.16, with Docker)**

In this section, we will demonstrate how to set up **SKOSMOS** using **Docker Compose**. This method allows for easy deployment of the SKOSMOS application along with its components, such as a triple store (like **Fuseki**) and caching mechanisms (such as **Varnish**). We assume you have an environment set up with Docker and Docker Compose installed.

**Assumptions for the environment:**
- You have Docker and Docker Compose installed on your system.
- You have a running instance of a triple store (GraphDB in this case) where the vocabularies are stored.
- We will be using the **GraphDB repository**: `https://vocabs.ilc4clarin.ilc.cnr.it/graphdb10/repositories/h2iosc_skosmos`.
- We will configure SKOSMOS to run at the following address (where `/skosmos` is the relative path):

`http(s)://{localhost|mydomain}/skosmos`

First, download version **2.16** of **SKOSMOS** from the official GitHub repository:
```bash
git clone https://github.com/NatLibFi/Skosmos.git -b 2.16
cd Skosmos
```

Navigate to the `dockerfiles` directory:
```bash
cd dockerfiles
```

**Docker Compose Configuration**

Here is an example **docker-compose.yml** file to deploy SKOSMOS along with **Apache Jena Fuseki** (as the triple store) and **Varnish** (as the caching mechanism). You can customize this configuration based on your setup.

```yaml
version: '3.7'

services:
  fuseki:
    container_name: skosmos-fuseki
    hostname: fuseki
    image: stain/jena-fuseki
    environment:
      - ADMIN_PASSWORD=admin
      - JVM_ARGS=-Xmx2g
    ports:
      - 9030:3030
    volumes:
      - type: bind
        source: ./config/skosmos.ttl
        target: /fuseki/configuration/skosmos.ttl

  fuseki-cache:
    container_name: skosmos-fuseki-cache
    hostname: fuseki-cache
    image: varnish
    ports:
      - 9031:80
    volumes:
      - type: bind
        source: ./config/varnish-default.vcl
        target: /etc/varnish/default.vcl

  skosmos:
    container_name: skosmos-web
    hostname: skosmos
    build:
      context: ..
      dockerfile: dockerfiles/Dockerfile.ubuntu
    ports:
      - 9090:80
    depends_on:
      - fuseki
      - fuseki-cache
    volumes:
      - type: bind
        source: ./config/config-docker-compose.ttl
        target: /var/www/html/config.ttl
```

**Explanation of Components**:
1. **Fuseki**: This service runs the **Apache Jena Fuseki** triple store, which stores and serves the SKOS vocabularies. We define the admin password and memory allocation using environment variables.
2. **Fuseki-Cache (Varnish)**: This service adds caching in front of the Fuseki instance to improve performance.
3. **SKOSMOS**: The main **SKOSMOS** web application that provides the user interface for browsing and querying the SKOS vocabularies.
   - **Container**: Hosts the **SKOSMOS** web application that serves the SKOS vocabularies and provides an interface for querying and browsing the vocabularies.
   - **Build**: The **build** context points to the root directory (`..`), meaning Docker will build SKOSMOS from the **Dockerfile** located in `dockerfiles/Dockerfile.ubuntu`.
   - **Ports**: Exposes port 9090 for accessing the SKOSMOS web interface.
   - **Volumes**: Binds a local configuration file `config-docker-compose.ttl` to the SKOSMOS container’s configuration at `/var/www/html/config.ttl`. This file contains the SKOSMOS-specific configuration, including the vocabularies.

**Customizing for Your Setup**

Since we already have a triple store in **GraphDB** (`https://vocabs.ilc4clarin.ilc.cnr.it/graphdb10/repositories/h2iosc_skosmos`), we can **comment out** the Fuseki and Fuseki-cache services in the Docker Compose file.

To modify the `docker-compose.yml` for our setup, simply comment out the sections for **Fuseki** and **Fuseki-cache** as shown below:

```yaml
# fuseki:
#   container_name: skosmos-fuseki
#   hostname: fuseki
#   image: stain/jena-fuseki
#   environment:
#     - ADMIN_PASSWORD=admin
#     - JVM_ARGS=-Xmx2g
#   ports:
#     - 9030:3030
#   volumes:
#     - type: bind
#       source: ./config/skosmos.ttl
#       target: /fuseki/configuration/skosmos.ttl

# fuseki-cache:
#   container_name: skosmos-fuseki-cache
#   hostname: fuseki-cache
#   image: varnish
#   ports:
#     - 9031:80
#   volumes:
#     - type: bind
#       source: ./config/varnish-default.vcl
#       target: /etc/varnish/default.vcl

skosmos:
  container_name: skosmos-web
  hostname: skosmos
  build:
    context: ..
    dockerfile: dockerfiles/Dockerfile.ubuntu
  ports:
    - 9090:80
  depends_on:
    # - fuseki
    # - fuseki-cache
  volumes:
    - type: bind
      source: ./config/config-docker-compose.ttl
      target: /var/www/html/config.ttl
```

In this setup:
- **Fuseki** and **Fuseki-cache** are disabled because we will connect SKOSMOS directly to the existing **GraphDB** triple store at `https://vocabs.ilc4clarin.ilc.cnr.it/graphdb10/repositories/h2iosc_skosmos`.

Now, you are ready to proceed with the SKOSMOS configuration. Next, we will explain how to configure the **config.ttl** file, define vocabularies, and link the **GraphDB** repository to SKOSMOS.

**Explanation of the Dockerfile.ubuntu**

This is the Dockerfile.ubuntu:

```
FROM ubuntu:20.04

LABEL maintainer="National Library of Finland"
LABEL version="0.1"
LABEL description="A Docker image for Skosmos with Apache httpd."

ARG DEBIAN_FRONTEND=noninteractive

# git is necessary for some composer packages e.g. davidstutz/bootstrap-multiselect
# gettext is necessary as php-gettext was available in 18.04, but not in 20.04
RUN apt-get update && apt-get install -y \
    apache2 \
    curl \
    gettext \
    git \
    libapache2-mod-php7.4 \
    locales \
    php7.4 \
    php7.4-curl \
    php7.4-xsl \
    php7.4-intl \
    php7.4-mbstring \
    php-apcu \
    php-zip \
    unzip \
 && rm -rf /var/lib/apt/lists/*

# https://stackoverflow.com/a/28406007
# fixes warnings like perl: warning: Setting locale failed.
RUN sed -i -e 's/# en_US.UTF-8 UTF-8/en_US.UTF-8 UTF-8/' /etc/locale.gen && \
    locale-gen \
        ar_AE.utf8 \
        da_DK.utf8 \
        de_DE.utf8 \
        en_GB.utf8 \
        en_US.utf8 \
        es_ES.utf8 \
        fa_IR.utf8 \
        fi_FI.utf8 \
        fr_FR.utf8 \
        it_IT.utf8 \
        nb_NO.utf8 \
        nl_NL.utf8 \
        nn_NO.utf8 \
        pl_PL.utf8 \
        pt_PT.utf8 \
        pt_BR.utf8 \
        ru_RU.utf8 \
        sv_SE.utf8 \
        zh_CN.utf8
ENV LANGUAGE=en_US:en  
ENV LC_ALL=en_US.UTF-8 
ENV LANG=en_US.UTF-8  

# timezone
RUN sed -i 's/;date.timezone =/date.timezone = "UTC"/g' /etc/php/7.4/apache2/php.ini

COPY dockerfiles/config/000-default.conf /etc/apache2/sites-available/000-default.conf

RUN a2enmod rewrite
RUN a2enmod expires

# set ServerName & redirect error log to stderr for docker logs
RUN echo "ServerName localhost" >> /etc/apache2/apache2.conf && \
        sed -ri \
        -e 's!^(\s*ErrorLog)\s+\S+!\1 /proc/self/fd/1!g' \
        "/etc/apache2/apache2.conf"

WORKDIR /var/www/html/skosmos
RUN rm ../index.html

# composer and packages layer
RUN curl -sS https://getcomposer.org/installer | php
COPY composer.json /var/www/html/skosmos

ENV COMPOSER_ALLOW_SUPERUSER=1

RUN php composer.phar install --no-dev --no-autoloader

# skosmos layer
COPY . /var/www/html/skosmos
RUN php composer.phar install --no-dev


# Configure Skosmos
COPY dockerfiles/config/config-docker.ttl /var/www/html/skosmos/config.ttl

#HEALTHCHECK --interval=5s --timeout=3s --retries=3 CMD curl -f http://localhost || exit 1

EXPOSE 80

CMD ["/usr/sbin/apache2ctl", "-D", "FOREGROUND"]

```

The Dockerfile builds an Ubuntu 20.04-based image designed to run SKOSMOS with Apache HTTP server, there are some keypoints:

1. **Apache Setup and Configuration:**
   - `COPY dockerfiles/config/000-default.conf /etc/apache2/sites-available/000-default.conf`: 
     - This line copies the Apache configuration file from the local machine (host) to the container at the path `/etc/apache2/sites-available/000-default.conf`. This Apache configuration file ensures that the web server is set up correctly for serving SKOSMOS.

2. **SKOSMOS Directory Setup:**
   - `WORKDIR /var/www/html/skosmos`: 
     - This sets the working directory inside the container to `/var/www/html/skosmos`. All subsequent commands will be executed in this directory.
   - `COPY . /var/www/html/skosmos`: 
     - This copies all files from the current directory on the host machine to the `/var/www/html/skosmos` directory inside the container. This is where SKOSMOS files, such as PHP scripts and assets, will be stored.
     - This setup ensures that SKOSMOS is correctly placed at `/var/www/html/skosmos`, which is essential for running the application on the specified path `/skosmos` when accessed via the web server.

3. **Configuration of SKOSMOS:**
   - `COPY dockerfiles/config/config-docker.ttl /var/www/html/skosmos/config.ttl`: 
     - This line copies the `config-docker.ttl` file from the host machine to the container, renaming it as `config.ttl` and placing it in the SKOSMOS directory `/var/www/html/skosmos`.
     - The `config.ttl` file is crucial as it defines the configuration settings for SKOSMOS, including the vocabulary definitions, SPARQL endpoint, and UI settings. It’s important that the file is placed in this exact path as required by SKOSMOS.

---

By following this configuration, SKOSMOS will be set up to run at the `/skosmos` path on the web server.

**Explanation of the Apache VirtualHost Configuration**

The provided Apache configuration file sets up a virtual host to serve SKOSMOS on port 80. Let’s break it down:

```apache
<VirtualHost *:80>
    #ServerName www.example.com
    ServerAdmin webmaster@localhost
    DocumentRoot /var/www/html/skosmos
    #LogLevel info ssl:warn
    LogFormat "%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-agent}i\"" combined
    ErrorLog /proc/self/fd/2
    CustomLog /proc/self/fd/1 combined
    <Directory /var/www/html/skosmos>
        Options Indexes FollowSymLinks MultiViews
        AllowOverride All
        Order allow,deny
        allow from all
    </Directory>
</VirtualHost>
```

In order to assign the correct path, we have to change these lines:

1. **`DocumentRoot /var/www/html/skosmos`**: 
   - This line defines the root directory where the website files are located. In this case, the SKOSMOS files are stored in `/var/www/html/skosmos`. Apache will serve files from this directory when the web server is accessed.

2. **`<Directory /var/www/html/skosmos>`**:
   - This section specifies settings for the `/var/www/html/skosmos` directory, where SKOSMOS files are stored. It controls how Apache interacts with the directory.

##### **Step 2: SKOSMOS Server Configuration**

In this step, we will configure the SKOSMOS application by modifying its configuration file (`config-docker-compose.ttl`). This file contains the settings for the SKOSMOS server, including the SPARQL endpoint, interface languages, and other parameters necessary for the application to function properly. Once you customize this configuration, it will be copied into the Docker container and renamed to `config.ttl`.

Here’s an explanation of the key configuration sections and how they need to be modified for your specific setup.

**Base SKOSMOS Configuration (`config-docker-compose.ttl`)**

The initial part of the configuration file defines server-level settings. Below is the sample `config-docker-compose.ttl` file with specific settings that need to be modified:

```turtle
@prefix void: <http://rdfs.org/ns/void#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix wv: <http://vocab.org/waiver/terms/norms> .
@prefix sd: <http://www.w3.org/ns/sparql-service-description#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix skosmos: <http://purl.org/net/skosmos#> .
@prefix isothes: <http://purl.org/iso25964/skos-thes#> .
@prefix mdrtype: <http://publications.europa.eu/resource/authority/dataset-type/> .
@prefix : <#> .

# Skosmos main configuration

:config a skosmos:Configuration ;
    # SPARQL endpoint
    skosmos:sparqlEndpoint <http://fuseki-cache:80/skosmos/sparql> ;
    # sparql-query extension, or "Generic" for plain SPARQL 1.1
    skosmos:sparqlDialect "JenaText" ;
    # whether to enable collation in sparql queries
    skosmos:sparqlCollationEnabled false ;
    # HTTP client configuration
    skosmos:sparqlTimeout 20 ;
    skosmos:httpTimeout 5 ;
    # customize the service name
    skosmos:serviceName "Skosmos" ;
    # customize the base element. Set this if the automatic base url detection doesn't work. For example setups behind a proxy.
    # skosmos:baseHref "http://localhost/Skosmos/" ;
    # interface languages available, and the corresponding system locales
    skosmos:languages (
        [ rdfs:label "ar" ; rdf:value "ar_AE.utf8" ]
        [ rdfs:label "da" ; rdf:value "da_DK.utf8" ]
        [ rdfs:label "de" ; rdf:value "de_DE.utf8" ]
        [ rdfs:label "en" ; rdf:value "en_GB.utf8" ]
        [ rdfs:label "en_US" ; rdf:value "en_US.utf8" ]
        [ rdfs:label "es" ; rdf:value "es_ES.utf8" ]
        [ rdfs:label "fa" ; rdf:value "fa_IR.utf8" ]
        [ rdfs:label "fi" ; rdf:value "fi_FI.utf8" ]
        [ rdfs:label "fr" ; rdf:value "fr_FR.utf8" ]
        [ rdfs:label "it" ; rdf:value "it_IT.utf8" ]
        [ rdfs:label "nb" ; rdf:value "nb_NO.utf8" ]
        [ rdfs:label "nl" ; rdf:value "nl_NL.utf8" ]
        [ rdfs:label "nn" ; rdf:value "nn_NO.utf8" ]
        [ rdfs:label "pl" ; rdf:value "pl_PL.utf8" ]
        [ rdfs:label "pt" ; rdf:value "pt_PT.utf8" ]
        [ rdfs:label "pt_BR" ; rdf:value "pt_BR.utf8" ]
        [ rdfs:label "ru" ; rdf:value "ru_RU.utf8" ]
        [ rdfs:label "sv" ; rdf:value "sv_SE.utf8" ]
        [ rdfs:label "zh" ; rdf:value "zh_CN.utf8" ]
    ) ;
    # how many results (maximum) to load at a time on the search results page
    skosmos:searchResultsSize 20 ;
    # how many items (maximum) to retrieve in transitive property queries
    skosmos:transitiveLimit 1000 ;
    # whether or not to log caught exceptions
    skosmos:logCaughtExceptions false ;
    # set to TRUE to enable logging into browser console
    skosmos:logBrowserConsole false ;
    # set to a logfile path to enable logging into log file
    # skosmos:logFileName "" ;
    # a default location for Twig template rendering
    skosmos:templateCache "/tmp/skosmos-template-cache" ;
    # customize the css by adding your own stylesheet
    skosmos:customCss "resource/css/stylesheet.css" ;
    # default email address where to send the feedback
    skosmos:feedbackAddress "" ;
    # email address to set as the sender for feedback messages
    skosmos:feedbackSender "" ;
    # email address to set as the envelope sender for feedback messages
    skosmos:feedbackEnvelopeSender "" ;
    # whether to display the ui language selection as a dropdown (useful for cases where there are more than 3 languages) 
    skosmos:uiLanguageDropdown true ;
    # whether to enable the spam honey pot or not, enabled by default
    skosmos:uiHoneypotEnabled true ;
    # default time a user must wait before submitting a form
    skosmos:uiHoneypotTime 5 ;
    # plugins to activate for the whole installation (including all vocabularies)
    skosmos:globalPlugins () .
```

**Explanation of Sections to Modify:**

1. **SPARQL Endpoint**
```turtle
skosmos:sparqlEndpoint <http://fuseki-cache:80/skosmos/sparql> ;
```
This line points SKOSMOS to the SPARQL endpoint where it can query vocabulary data. In our setup, we need to point this to the actual GraphDB repository instead of the default Fuseki endpoint.

- **Update**: Replace this with your GraphDB endpoint:

```turtle
skosmos:sparqlEndpoint <https://vocabs.ilc4clarin.ilc.cnr.it/graphdb10/repositories/h2iosc_skosmos> ;
```

2. **SPARQL Dialect**
```turtle
skosmos:sparqlDialect "JenaText" ;
```
This setting defines the SPARQL dialect used by the endpoint. By default, it is set to "JenaText" for Fuseki's text indexing. Since we are using GraphDB, we should change this to "Generic" for standard SPARQL 1.1.

- **Update**: Change to `Generic`:

```turtle
skosmos:sparqlDialect "Generic" ;
```

**Service Name**
```turtle
skosmos:serviceName "Skosmos" ;
```
This is the display name that appears at the top of the SKOSMOS user interface. You can customize it based on your project.

- **Update**: Set your own service name. For example:

```turtle
skosmos:serviceName "ILC4CLARIN Vocabularies" ;
```

**Base URL**
```turtle
# skosmos:baseHref "http://localhost/Skosmos/" ;
```
The `baseHref` defines the base URL for the SKOSMOS service. If SKOSMOS is behind a proxy or uses a custom URL, you'll need to uncomment this and set the correct base URL.

- **Update**: Uncomment and set your domain with `/skosmos` at the end. For example:

```turtle
skosmos:baseHref "https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/" ;
```

5. **Language Settings**
```turtle
skosmos:languages (
    [ rdfs:label "ar" ; rdf:value "ar_AE.utf8" ]
    [ rdfs:label "da" ; rdf:value "da_DK.utf8" ]
    ...
) ;
```
The `skosmos:languages` section defines the user interface languages and their corresponding system locales. You will modify this to match the languages in your vocabularies. Below, we will generate the list of languages based on the `prefLabels`, `altLabels`, and `note` fields in your vocabulary data.

**Script to Extract Unique Languages from Vocabularies**

The following Python script scans your vocabulary files in the `input_data` directory and extracts all unique language codes from `prefLabels`, `altLabels`, and `note` fields:

In [2]:
import os
import json

def extract_languages_from_vocab(directory):
    lang_set = set()

    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            with open(os.path.join(directory, filename), "r", encoding="utf-8") as f:
                data = json.load(f)
                for item in data:
                    # Extract languages from prefLabels if they are not None
                    if 'prefLabels' in item and item['prefLabels'] is not None:
                        for label in item['prefLabels']:
                            if label.get('lang'):
                                lang_set.add(label['lang'].lower())

                    # Extract languages from altLabels if they are not None
                    if 'altLabels' in item and item['altLabels'] is not None:
                        for label in item['altLabels']:
                            if label.get('lang'):
                                lang_set.add(label['lang'].lower())

                    # Extract languages from note if they are not None
                    if 'note' in item and item['note'] is not None:
                        for note in item['note']:
                            if note.get('lang'):
                                lang_set.add(note['lang'].lower())

    return sorted(lang_set)

# Directory containing input JSON vocabulary files
input_data_dir = "input_data"

# Get unique languages
languages = extract_languages_from_vocab(input_data_dir)

# Print the language list in the format needed for SKOSMOS
for lang in languages:
    print(f'rdfs:label "{lang}"')


rdfs:label "ca"
rdfs:label "en"
rdfs:label "es"
rdfs:label "es-ar"
rdfs:label "es-mx"
rdfs:label "fr"
rdfs:label "gl"
rdfs:label "it"
rdfs:label "pt"
rdfs:label "pt-br"
rdfs:label "ro"


For rdf:value it's 

For rdf:value is **MANDATORY** to check the locales list for corresponding language by typing this command to your UNIX machine `$ locale -a` or by searching in some sites that have a list of all supported locales. If there are some missing locales, you have to install these on the Skosmos Ubuntu Server.

##### **Step 3: SKOSMOS Vocabularies Configuration**

In this step, we will explain how to modify the `config-docker-compose.ttl` file to include the vocabularies that have been loaded into GraphDB. This step ensures that the vocabularies are displayed and accessible through SKOSMOS. First, let’s break down a base vocabulary configuration already present in SKOSMOS during the installation.

**Example Vocabulary Configuration**

```ttl
:unesco a skosmos:Vocabulary, void:Dataset ;
    dc:title "UNESCO Thesaurus"@en ;
    skosmos:shortName "UNESCO";
    dc:subject :cat_general ;
    void:uriSpace "http://skos.um.es/unescothes/";
    skosmos:language "en", "es", "fr", "ru";
    skosmos:defaultLanguage "en";
    skosmos:showTopConcepts true ;
    skosmos:fullAlphabeticalIndex true ;
    skosmos:groupClass isothes:ConceptGroup ;
    void:sparqlEndpoint <http://fuseki-cache:80/skosmos/sparql> ;
    skosmos:sparqlGraph <http://skos.um.es/unescothes/> .
```

**Explanation of Key Properties**

- **`:unesco`**: This is the URI of the vocabulary being defined.
- **`dc:title "UNESCO Thesaurus"@en`**: The title of the vocabulary in English.
- **`skosmos:shortName "UNESCO"`**: A shorter, human-readable name for the vocabulary.
- **`dc:subject :cat_general`**: Specifies the subject or category of the vocabulary.
- **`void:uriSpace "http://skos.um.es/unescothes/"`**: The base URI for all concepts in this vocabulary.
- **`skosmos:language "en", "es", "fr", "ru"`**: Defines the languages available in the vocabulary.
- **`skosmos:defaultLanguage "en"`**: The default language for the vocabulary.
- **`skosmos:showTopConcepts true`**: Whether to show the top concepts of the vocabulary.
- **`skosmos:fullAlphabeticalIndex true`**: Enables a full alphabetical index of all concepts.
- **`void:sparqlEndpoint <http://fuseki-cache:80/skosmos/sparql>`**: The SPARQL endpoint for querying the vocabulary.
- **`skosmos:sparqlGraph <http://skos.um.es/unescothes/>`**: The named graph in the triple store where this vocabulary is stored.

**Creating a Category for Pan-Latin Lexicons**

To group all Pan-Latin lexicons (Realiter OTPL), we need to define a custom category. First, we create a custom prefix:

```ttl
@prefix panlatin: <http://vocabs.ilc4clarin.ilc.cnr.it/panlatinlexicon> .
```

Next, we define the category as a `skos:Concept`:

```ttl
panlatin:LexiconCat a skos:Concept ;
    skos:prefLabel "REALITER - OTPL Pan-Latin Lexicons"@it,
        "REALITER - OTPL Pan-Latin Lexicons"@en ;
    skos:definition "Category for grouping the Pan-Latin lexicons section"@en .
```

This defines the category with labels in both Italian and English, allowing us to group lexicons under this concept. 

---

**Dynamically Generating Vocabularies Configuration**

Next, we will generate a `.ttl` file to configure vocabularies in SKOSMOS based on the files present in the `concept_schemes` directory. The configuration will be copied to `config-docker-compose.ttl`. 

Here’s how the vocabularies will be structured:

**Vocabulary Definition Structure**
For each vocabulary:
- The vocabulary's URI will be formed as: `:pl_{filename_without_extension}`.
- It will reference the relevant metadata from the JSON files.

**Configuration Example**

```ttl
@prefix void: <http://rdfs.org/ns/void#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix wv: <http://vocab.org/waiver/terms/norms> .
@prefix sd: <http://www.w3.org/ns/sparql-service-description#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix skosmos: <http://purl.org/net/skosmos#> .
@prefix isothes: <http://purl.org/iso25964/skos-thes#> .
@prefix mdrtype: <http://publications.europa.eu/resource/authority/dataset-type/> .
@prefix : <#> .
@prefix panlatin: <http://vocabs.ilc4clarin.ilc.cnr.it/panlatinlexicon> .

:pl_20_500_11752_OPEN_1014 a skosmos:Vocabulary, void:Dataset ;
    dc:title "Pan-Latin Office Supplies Vocabulary"@en ;
    skosmos:shortName "Pan-Latin Office Supplies Vocabulary" ;
    dc:subject panlatin:LexiconCat ;
    void:uriSpace "https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/pl_20_500_11752_OPEN_1014/" ;
    skosmos:language "it", "en", "fr", "es-AR", "pt-BR" ;
    skosmos:defaultLanguage "it" ;
    void:sparqlEndpoint <https://vocabs.ilc4clarin.ilc.cnr.it/graphdb10/repositories/h2iosc_skosmos> ;
    skosmos:sparqlGraph <https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/pl_20_500_11752_OPEN_1014/> .
```

**Explanation of Properties:**
- **`:pl_20_500_11752_OPEN_1014`**: The URI of the vocabulary, based on the filename.
- **`dc:title`**: The title of the vocabulary, extracted from the `dc_title` in the JSON file, assigned the language tag `@en`.
- **`skosmos:shortName`**: A shorter name derived from `dc_title` without the language tag.
- **`dc:subject`**: Assigns the vocabulary to the `panlatin:LexiconCat` category.
- **`void:uriSpace`**: The URI space for all concepts in this vocabulary.
- **`skosmos:language`**: Lists all available languages, transforming language codes where necessary (e.g., `es-AR`, `pt-BR`).
- **`skosmos:defaultLanguage`**: Sets the default language to `it`.
- **`void:sparqlEndpoint`**: The SPARQL endpoint where the vocabulary can be queried (GraphDB in this case).
- **`skosmos:sparqlGraph`**: The named graph where the vocabulary is stored.

**Script to Generate Vocabulary Configuration**

Here is a Python script that generates this vocabulary configuration file dynamically:

In [4]:
import os
import json

def extract_languages_from_vocab(directory, filename):
    """
    Function to extract unique languages from prefLabels, altLabels, and note fields
    in the JSON file corresponding to the given filename in a given directory.
    """
    lang_set = set()
    file_path = os.path.join(directory, filename)

    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            for item in data:
                # Extract languages from prefLabels if they are not None
                if 'prefLabels' in item and item['prefLabels'] is not None:
                    for label in item['prefLabels']:
                        if label.get('lang'):
                            lang_set.add(label['lang'].lower())

                # Extract languages from altLabels if they are not None
                if 'altLabels' in item and item['altLabels'] is not None:
                    for label in item['altLabels']:
                        if label.get('lang'):
                            lang_set.add(label['lang'].lower())

                # Extract languages from note if they are not None
                if 'note' in item and item['note'] is not None:
                    for note in item['note']:
                        if note.get('lang'):
                            lang_set.add(note['lang'].lower())

    return sorted(lang_set)

def generate_vocab_config(concept_schemes_dir, input_files_dir, output_file):
    # List of static prefixes
    prefixes = """
@prefix void: <http://rdfs.org/ns/void#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix wv: <http://vocab.org/waiver/terms/norms> .
@prefix sd: <http://www.w3.org/ns/sparql-service-description#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix skosmos: <http://purl.org/net/skosmos#> .
@prefix isothes: <http://purl.org/iso25964/skos-thes#> .
@prefix : <#> .
@prefix panlatin: <http://vocabs.ilc4clarin.ilc.cnr.it/panlatinlexicon> .
"""

    with open(output_file, "w", encoding="utf-8") as out_f:
        out_f.write(prefixes + "\n\n")

        # Iterate over JSON files in concept_schemes_dir
        for filename in os.listdir(concept_schemes_dir):
            if filename.endswith(".json"):
                vocab_name = filename.replace(".json", "")
                
                # Check if corresponding file exists in input_data directory
                input_data_file = f"{vocab_name}.json"
                if os.path.exists(os.path.join(input_files_dir, input_data_file)):
                    json_path = os.path.join(concept_schemes_dir, filename)
                    
                    # Load the concept schema JSON data
                    with open(json_path, "r", encoding="utf-8") as f:
                        data = json.load(f)
                        title = data.get("dc_title", "Unknown Title")
                    
                    # Extract languages from the corresponding input file
                    languages = extract_languages_from_vocab(input_files_dir, input_data_file)
                    
                    # Format the languages correctly (uppercase second part, if present)
                    lang_str = ", ".join([f'"{lang.split("-")[0]}-{lang.split("-")[1].upper()}"' if "-" in lang else f'"{lang}"' for lang in languages])

                    # Generate the vocabulary configuration block
                    vocab_config = f"""
:pl_{vocab_name} a skosmos:Vocabulary, void:Dataset ;
    dc:title "{title}"@en ;
    skosmos:shortName "{title}" ;
    dc:subject panlatin:LexiconCat ;
    void:uriSpace "https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/pl_{vocab_name}/" ;
    skosmos:language {lang_str} ;
    skosmos:defaultLanguage "it" ;
    skosmos:showTopConcepts "true" ;
    void:sparqlEndpoint <https://vocabs.ilc4clarin.ilc.cnr.it/graphdb10/repositories/h2iosc_skosmos> ;
    skosmos:sparqlGraph <https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/pl_{vocab_name}/> .
"""
                    out_f.write(vocab_config)
                else:
                    print(f"Skipping {vocab_name}: no matching file in input_data.")

# Example usage
generate_vocab_config("concept_schemes", "input_data", "vocabs_data.ttl")

Skipping 20_500_11752_OPEN_1008: no matching file in input_data.
Skipping 20_500_11752_OPEN_1006: no matching file in input_data.
Skipping 20_500_11752_OPEN_994: no matching file in input_data.
Skipping 20_500_11752_OPEN_995: no matching file in input_data.
Skipping 20_500_11752_OPEN_975: no matching file in input_data.
Skipping 20_500_11752_OPEN_987: no matching file in input_data.
Skipping 20_500_11752_OPEN_993: no matching file in input_data.


In summary, the vocabulary information (like `:pl_{vocab_name} a skosmos:Vocabulary, void:Dataset;`) should be added to the `config-docker-compose.ttl` file, while ensuring that the prefix declarations (`@prefix ...`) are excluded since they are already present in the configuration file. Restart the docker container after edit.

##### **Step 4: Configuring Apache for Content Negotiation (RewriteRule)**

In this step, we need to configure Apache to correctly handle requests for URIs under `/vocabularies/`, ensuring they are properly redirected to the SKOSMOS interface for rendering in HTML. The goal is to make sure that resource URIs resolve and display properly through SKOSMOS when requested by a web browser.

We assume that:

- We have a remote Apache server acting as a reverse proxy.
- Requests to `/skosmos` are routed to a Docker container running the SKOSMOS application.
- The SKOSMOS application within Docker is configured to render SKOS data as HTML.

**Apache Configuration for Content Negotiation**

To achieve this, we need to add a **RewriteRule** that directs requests for resources under `/vocabularies/` to SKOSMOS using content negotiation. This ensures that URIs such as `https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/` will be properly resolved and displayed via SKOSMOS.

We will add the following rule to the Apache configuration file:

```apache
RewriteRule ^/vocabularies/(.*)$ /skosmos/entity?uri=https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/$1 [R=302,L]
```

This rule captures any request for `/vocabularies/{resource}` and redirects it to the SKOSMOS `/entity` endpoint, appending the original URI as a parameter. 

**Where to Place the RewriteRule**

Typically, this rewrite rule should be placed in one of the following files:

- **httpd-vhost.conf**: If you're using virtual hosts for Apache configuration.
- **httpd-ssl.conf**: If the site is served over HTTPS, this file often handles SSL configuration.
- **Custom config file**: Any additional configuration file included in `httpd.conf` can also be used.

Make sure that this file is included within the `httpd.conf` file, which is the primary Apache configuration file.

**Example VirtualHost Configuration for Reverse Proxy**

Here's an example configuration for a VirtualHost using SSL, which redirects `/vocabularies/` requests to SKOSMOS:

```apache
<VirtualHost *:443>
    ServerName vocabs.ilc4clarin.ilc.cnr.it

    SSLEngine on
    SSLCertificateFile /etc/letsencrypt/live/vocabs.ilc4clarin.ilc.cnr.it/fullchain.pem
    SSLCertificateKeyFile /etc/letsencrypt/live/vocabs.ilc4clarin.ilc.cnr.it/privkey.pem
    Include /etc/letsencrypt/options-ssl-apache.conf

    RewriteEngine On
    ProxyRequests Off
    ProxyPreserveHost On

    RequestHeader set X-Forwarded-Proto "https"
    RequestHeader set X-Forwarded-Port "443"

    # Rule to rewrite requests for /vocabularies/ to SKOSMOS
    RewriteRule ^/vocabularies/(.*)$ /skosmos/entity?uri=https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/$1 [R=302,L]

    # Proxy settings to route /skosmos to SKOSMOS application in Docker
    <Location "/skosmos">
       ProxyPass http://skosmos_web/  # Or IP address of the machine running SKOSMOS
       ProxyPassReverse http://skosmos_web/
    </Location>

    <Proxy *>
        Order deny,allow
        Allow from all
    </Proxy>
</VirtualHost>
```

**Explanation of Key Components:**

- **RewriteRule**: The rewrite rule captures any request made to `/vocabularies/` and redirects it to SKOSMOS’s `/entity` endpoint, passing the requested URI (`$1`) as a query parameter (`uri`). The `[R=302,L]` flags ensure that the request is redirected with a 302 (temporary) redirect and that no further processing is done for this request.

- **ProxyPass**: The `ProxyPass` and `ProxyPassReverse` directives are used to pass the `/skosmos` requests to the internal SKOSMOS service (running in Docker or another server). This ensures that requests for SKOSMOS are routed correctly through the reverse proxy.

- **RequestHeader**: These headers ensure that the original request scheme (`https`) and port are forwarded correctly, so SKOSMOS is aware it's being accessed over HTTPS.

- **SSL Configuration**: The SSL certificate and key files (`SSLCertificateFile` and `SSLCertificateKeyFile`) ensure that the connection is secured over HTTPS. You will need to set these to point to your actual SSL certificate files.

By configuring Apache with these settings, the URI `https://vocabs.ilc4clarin.ilc.cnr.it/vocabularies/` will now point to the SKOSMOS `/entity` endpoint, where resources will be rendered properly in HTML when accessed via a web browser. Restart Apache after edit.

##### **Step 5: Fixing Translations**

It can sometimes happen that certain parts of SKOSMOS are not rendered when switching languages due to missing translation information. To resolve this, it's necessary to edit the translation files located in the `/resource/translations` directory. These files use the `.po` format and define the translated text for various user interface components.

**Example Structure of a `.po` File:**

Here is an example of a `.po` file that handles translations in SKOSMOS:

```plaintext
# 
# Translators:
# Armando Stellato <stellato@info.uniroma2.it>, 2015-2016
# osma <osma.suominen@helsinki.fi>, 2015,2017,2020
# Yksityis Yrittäjä <okko.vainonen@helsinki.fi>, 2021
msgid ""
msgstr ""
"Project-Id-Version: Skosmos\n"
"Report-Msgid-Bugs-To: \n"
"POT-Creation-Date: 2017-01-16 10:26+0200\n"
"PO-Revision-Date: 2015-06-18 07:06+0000\n"
"Last-Translator: Yksityis Yrittäjä <okko.vainonen@helsinki.fi>, 2021\n"
"Language-Team: Italian (http://www.transifex.com/national-library-of-finland/skosmos/language/it/)\n"
"MIME-Version: 1.0\n"
"Content-Type: text/plain; charset=UTF-8\n"
"Content-Transfer-Encoding: 8bit\n"
"Language: it\n"
"Plural-Forms: nplurals=3; plural=n == 1 ? 0 : n != 0 && n % 1000000 == 0 ? 1 : 2;\n"
"X-Generator: Poedit 1.8.7.1\n"
"X-Poedit-Basepath: ../..\n"
"X-Poedit-SearchPath-0: view\n"
"X-Poedit-SearchPath-1: controller\n"
"X-Poedit-SearchPath-2: model\n"
"X-Poedit-SourceCharset: utf-8\n"

msgid "Download this vocabulary:"
msgstr ""

#: controller/Controller.php:38
#: /tmp/cache/587c83c09abaa/4e/4e05e5051b20c875609479922a558d59b8500ae386eaf79bc460e69d8781a753.php:21
msgid "in_this_language"
msgstr "in Italiano"

#: controller/RestController.php:271
msgid "skos:Concept"
msgstr "Concetto"

#: controller/RestController.php:280
msgid "skos:Collection"
msgstr "Collezione"

#: model/Concept.php:455
msgid "skosmos:created"
msgstr "Creato"

#: model/Concept.php:460
msgid "skosmos:modified"
msgstr "ultima modifica"

#: model/VocabularyCategory.php:39
msgid "Vocabularies"
msgstr "Vocabolari"
...

**How to Fix Missing Translations**

When parts of the SKOSMOS interface are not being translated, you should:

1. **Locate the translation file**: Identify the relevant `.po` file in the `/resource/translations` directory for the language in question.
2. **Add missing translation strings**: If you notice any untranslated text or "msgstr" fields that are empty (`""`), fill in the correct translations.
3. **Ensure accuracy**: The `msgid` is the original English string, and the `msgstr` is where the translated text should be provided. You can refer to other translations or tools like [Transifex](https://www.transifex.com) if needed.

**Example of Adding Missing Translations:**

If the message ID `Download this vocabulary:` is missing its Italian translation, you would update the `.po` file as follows:

```plaintext
msgid "Download this vocabulary:"
msgstr "Scarica questo vocabolario:"
```

**Compiling the Translations**

Once the translations have been edited, you must compile them using a special script inside the Docker container.

1. **Make the `compile-translations` file executable**: You can set the necessary permissions with the following command in your Docker container:
   
   ```bash
   chmod +x compile-translations
   ```

2. **Run the compile-translations script**: This command will compile the `.po` files into `.mo` files (binary format) that SKOSMOS can read.

   ```bash
   ./compile-translations
   ```

3. **Restart the SKOSMOS container**: After compiling the translations, restart your Docker container to apply the changes:

   ```bash
   docker-compose restart skosmos
   ```

This process ensures that the updated translations are recognized and displayed in the SKOSMOS user interface.

##### **Step 6: Using SKOSMOS REST API**

Once the vocabulary data is successfully uploaded to the SKOSMOS platform, you can access and retrieve data through SKOSMOS' powerful REST API. The API provides endpoints for querying the vocabularies, retrieving concept information, and managing language settings, among other functions. You can consult the full documentation of the SKOSMOS API at [SKOSMOS API Documentation](https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/doc/).

Here are some of the most useful SKOSMOS REST API endpoints:

#### 1. **List All Vocabularies**
   - **Endpoint**: `/rest/v1/vocabularies`
   - **Description**: Retrieves a list of all vocabularies available on the SKOSMOS instance, with details such as title, language, and subject.
   - **Example**: 
     ```bash
     curl https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/rest/vocabularies
     ```

#### 2. **Search for Concepts in a Vocabulary**
   - **Endpoint**: `/rest/v1/vocabulary/{vocab}/search`
   - **Description**: Allows you to search for a term in a specific vocabulary. You can search concepts by label, definition, or notation.
   - **Parameters**: `query`, `type`, `lang`, `parent`, `group`
   - **Example**:
     ```bash
     curl https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/rest/vocabulary/{vocab}/search?query=water
     ```

#### 3. **Get Concept Information**
   - **Endpoint**: `/rest/v1/vocabulary/{vocab}/concept`
   - **Description**: Fetches detailed information about a specific concept within a vocabulary, including labels, broader/narrower relationships, and definitions.
   - **Parameters**: `uri`, `lang`
   - **Example**:
     ```bash
     curl https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/rest/vocabulary/{vocab}/concept?uri={conceptURI}
     ```

#### 4. **Get Top Concepts**
   - **Endpoint**: `/rest/v1/vocabulary/{vocab}/topConcepts`
   - **Description**: Retrieves all top-level concepts from a vocabulary. Top concepts are usually broad categories or entry points for a vocabulary.
   - **Parameters**: `lang`
   - **Example**:
     ```bash
     curl https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/rest/vocabulary/{vocab}/topConcepts
     ```

#### 5. **Get Concept Label**
   - **Endpoint**: `/rest/v1/vocabulary/{vocab}/label`
   - **Description**: Searches for concepts by their labels, returning concepts that match the search query.
   - **Parameters**: `query`, `lang`, `type`
   - **Example**:
     ```bash
     curl https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/rest/vocabulary/{vocab}/label?query=water
     ```

#### 6. **Retrieve Concept Groups**
   - **Endpoint**: `/rest/v1/vocabulary/{vocab}/groups`
   - **Description**: Fetches the concept groups within a vocabulary, useful for hierarchical browsing or grouped collections.
   - **Parameters**: `lang`
   - **Example**:
     ```bash
     curl https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/rest/vocabulary/{vocab}/groups
     ```

#### 7. **Get Languages Available for a Vocabulary**
   - **Endpoint**: `/rest/v1/vocabulary/{vocab}/languages`
   - **Description**: Retrieves a list of all languages in which a given vocabulary is available.
   - **Example**:
     ```bash
     curl https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/rest/vocabulary/{vocab}/languages
     ```

#### 8. **Get All Concepts Under a Specific Concept Scheme**
   - **Endpoint**: `/rest/v1/vocabulary/{vocab}/concepts`
   - **Description**: Lists all concepts that are part of a specific concept scheme or collection.
   - **Parameters**: `lang`, `group`
   - **Example**:
     ```bash
     curl https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/rest/vocabulary/{vocab}/concepts?lang=en&group=someGroupURI
     ```

#### 9. **Get All Data for a Concept as JSON-LD**
   - **Endpoint**: `/rest/v1/vocabulary/{vocab}/data`
   - **Description**: Provides all available data for a specific concept in JSON-LD format.
   - **Parameters**: `uri`, `format=jsonld`
   - **Example**:
     ```bash
     curl https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/rest/vocabulary/{vocab}/data?uri={conceptURI}&format=jsonld
     ```

#### 10. **Retrieve the Alphabetical Index**
   - **Endpoint**: `/rest/v1/vocabulary/{vocab}/index`
   - **Description**: Returns the alphabetical index for a given vocabulary, useful for navigating concepts.
   - **Parameters**: `lang`, `letter`, `limit`, `offset`
   - **Example**:
     ```bash
     curl https://vocabs.ilc4clarin.ilc.cnr.it/skosmos/rest/vocabulary/{vocab}/index?lang=en&letter=A
     ```

These APIs provide a comprehensive way to interact programmatically with the vocabularies hosted on SKOSMOS. They allow users to search, retrieve, and explore the semantic relationships between concepts in a flexible and efficient manner.